# **SVM**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 4 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Project-V4/feature-extraction/NotAnnotated/AveragePooled/split_features_4s_all.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[1.7294491e-05, 1.8239583e-05, 1.9485902e-05, ..., 5.1940111e-07,
         3.2287385e-08, 2.7940120e-08],
        [7.9314253e-07, 8.0714307e-07, 8.3096978e-07, ..., 1.3027967e-06,
         8.4759665e-07, 8.0172282e-07],
        [3.0612518e-06, 3.0607123e-06, 3.0592107e-06, ..., 6.0072955e-07,
         3.6999687e-08, 1.4126979e-08],
        ...,
        [1.0815166e-06, 1.0910663e-06, 1.1071655e-06, ..., 4.2367597e-07,
         1.3173792e-07, 1.2475495e-07],
        [2.1907644e-07, 2.7223422e-07, 3.6665881e-07, ..., 5.7446897e-07,
         1.8625977e-07, 1.7997159e-07],
        [2.4198832e-06, 2.6966893e-06, 3.1864115e-06, ..., 1.2659899e-05,
         2.3157411e-07, 2.2612821e-07]], dtype=float32),
 'mfcc': array([[-1.0000000e+00, -2.3626368e-01, -5.0428110e-01, ...,
          5.1737577e-03,  1.1148715e-02,  3.4610374e-04],
        [-1.0000000e+00, -2.4848318e-01, -4.7058651e-01, ...,
          2.9826174e-03,  9.7121680e-03, -5.3395964e-03],
        [-1.0000000e

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(10981,)

array([ 0,  2, 12, 19,  1, 16, 12, 12,  2, 10,  1, 17, 17, 16,  5],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(10981, 60)

array([2.30762816e-05, 2.34475592e-05, 2.40821773e-05, 2.51011024e-05,
       2.66466941e-05, 2.92958284e-05, 3.51628805e-05, 6.25891989e-05,
       1.92321357e-04, 7.03408732e-04, 2.65689683e-03, 9.06991586e-03,
       2.57462058e-02, 3.97482254e-02, 6.07631765e-02, 8.05736035e-02,
       1.04595460e-01, 1.15686581e-01, 1.57870665e-01, 1.72645211e-01,
       1.86443135e-01, 1.28887072e-01, 8.85766745e-02, 7.87886903e-02,
       9.79994535e-02, 7.95486793e-02, 9.88657176e-02, 1.29356623e-01,
       1.11660235e-01, 1.01738557e-01, 8.50578472e-02, 9.85676572e-02,
       2.73807824e-01, 2.09708124e-01, 1.23314388e-01, 1.02081485e-01,
       3.67147297e-01, 4.48260367e-01, 2.47974992e-01, 2.81487495e-01,
       1.71025455e-01, 1.56875506e-01, 9.04699489e-02, 5.01980409e-02,
       1.14354856e-01, 1.35187045e-01, 8.44150558e-02, 7.01157376e-02,
       4.45818938e-02, 2.53626164e-02, 1.44096855e-02, 1.17013389e-02,
       1.02886660e-02, 6.78899651e-03, 3.07482062e-03, 6.01989741e-04,
      

'mfcc'

(10981, 20)

...

(10981, 12)

array([0.18950042, 0.2706272 , 0.45278975, 0.28500038, 0.22375982,
       0.26610336, 0.55544305, 0.43051782, 0.378617  , 0.37237042,
       0.28450966, 0.24387611], dtype=float32)

'rms'

(10981, 1)

array([0.02751933], dtype=float32)

In [13]:
display(val_y.shape)
display(val_y[:15])

(2928,)

array([ 1,  0,  3, 13, 15,  7,  0, 11, 17, 18, 10,  0,  1,  2,  2],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(2928, 60)

array([1.46651128e-05, 1.48556392e-05, 1.51859913e-05, 1.56665410e-05,
       1.63160948e-05, 1.78167320e-05, 1.98534417e-05, 4.05498686e-05,
       1.59472402e-04, 6.66950422e-04, 2.94189877e-03, 8.73562507e-03,
       2.48509813e-02, 5.62934652e-02, 1.06700696e-01, 1.37720481e-01,
       1.56773552e-01, 1.66458443e-01, 1.62727058e-01, 1.53277159e-01,
       1.42315477e-01, 1.38097569e-01, 1.40576839e-01, 1.23165958e-01,
       1.06688209e-01, 8.76584128e-02, 8.31917152e-02, 8.77315849e-02,
       9.44868550e-02, 8.88955891e-02, 7.43178725e-02, 8.85595381e-02,
       1.43550202e-01, 1.52272820e-01, 1.52132839e-01, 1.91071063e-01,
       2.06672847e-01, 1.97658002e-01, 2.46173069e-01, 2.55545259e-01,
       1.99123114e-01, 1.60002843e-01, 1.69800088e-01, 2.38350451e-01,
       2.02516481e-01, 2.13135630e-01, 9.99766588e-02, 1.05907999e-01,
       7.79003873e-02, 4.98168357e-02, 2.12974120e-02, 1.53250685e-02,
       6.21610088e-03, 3.42591107e-03, 1.57294865e-03, 2.99405801e-04,
      

'mfcc'

(2928, 20)

...

(2928, 12)

array([0.3695338 , 0.37865743, 0.4104207 , 0.31655085, 0.3333765 ,
       0.34385303, 0.2945401 , 0.33566204, 0.36030295, 0.3360909 ,
       0.29794866, 0.2928751 ], dtype=float32)

'rms'

(2928, 1)

array([0.03157331], dtype=float32)

## **SVM Model**

In [15]:
train_results = {}
val_results = {}

### **With all the features**

In [16]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['rms'], train_features['melspectrogram']), axis=1)

training_features.shape

(10981, 93)

In [17]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['rms'], val_features['melspectrogram']), axis=1)

validation_features.shape

(2928, 93)

Fit the model with training data

We can tune a lot of the parameters to prevent the overfitting of the model.
I saw that you could use sklearn.model_selection.GridSearchCV for hyperparameter tuning.
You cant really improve an overfitted model fr by hyperparameter tuning. I can try to.

We are already using 100 estimaters by default, and even if we add more, the model can not learn anything useful after that

To improve the performance, we probably need a more complicated algorithm, or more traaining data

In [21]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [22]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 0.8645842819415354
Validation accuracy: 0.41222677595628415


In [23]:
train_yhat_result = model.predict(training_features)

print('Training classification Report \n')
print(classification_report(train_y, train_yhat_result))

Training classification Report 

              precision    recall  f1-score   support

           0       0.77      0.93      0.84      1542
           1       0.83      0.90      0.86      1114
           2       0.88      0.83      0.85       763
           3       0.95      0.91      0.93       708
           4       0.94      0.81      0.87       105
           5       0.95      0.91      0.93       177
           6       0.85      0.53      0.65       199
           7       0.89      0.85      0.87       630
           8       0.83      0.47      0.60       175
           9       0.95      0.89      0.92       314
          10       0.85      0.87      0.86       437
          11       0.87      0.74      0.80       168
          12       0.87      0.91      0.89      1098
          13       0.96      0.86      0.91       291
          14       0.87      0.83      0.85       424
          15       0.93      0.82      0.87       629
          16       0.97      0.93      0.95     

In [24]:
val_yhat_result = model.predict(validation_features)

print('Validation classification Report \n')
print(classification_report(val_y, val_yhat_result))

Validation classification Report 

              precision    recall  f1-score   support

           0       0.38      0.65      0.48       412
           1       0.20      0.35      0.25       184
           2       0.81      0.34      0.48       717
           3       0.49      0.87      0.62       147
           4       0.20      0.03      0.05        74
           5       0.59      0.63      0.61        30
           6       0.10      0.10      0.10        29
           7       0.56      0.40      0.47       172
           8       0.29      0.42      0.34        59
           9       0.64      0.74      0.69       100
          10       0.36      0.37      0.36       117
          11       0.06      0.03      0.04        59
          12       0.21      0.27      0.24       139
          13       0.00      0.00      0.00        51
          14       0.06      0.17      0.09        30
          15       0.59      0.32      0.41       211
          16       0.32      0.14      0.19   

### **With MFCCs only**

In [25]:
training_features = train_features['mfcc']

training_features.shape

(10981, 20)

In [26]:
validation_features = val_features['mfcc']

validation_features.shape

(2928, 20)

In [27]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [28]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 0.5877424642564429
Validation accuracy: 0.3849043715846995


## **With MFCCs and Mel-Spectrogram**

In [29]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(10981, 80)

In [30]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(2928, 80)

In [31]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [32]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 0.8836171569073855
Validation accuracy: 0.3989071038251366


## **With MFCCs and Chroma**

In [33]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(10981, 32)

In [34]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(2928, 32)

In [35]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [36]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 0.6607777069483654
Validation accuracy: 0.37739071038251365


## **With MFCCs and RMS**

In [37]:
training_features = np.concatenate((train_features['mfcc'], train_features['rms']), axis=1)

training_features.shape

(10981, 21)

In [38]:
validation_features = np.concatenate((val_features['mfcc'], val_features['rms']), axis=1)

validation_features.shape

(2928, 21)

In [39]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [40]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_rms'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_rms'] = model.score(validation_features, val_y)

Training accuracy: 0.5953009744103451
Validation accuracy: 0.398224043715847


## **With Mel-Spectrogram, MFCCs, RMS**

In [41]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['rms'], train_features['mfcc']), axis=1)

training_features.shape

(10981, 81)

In [42]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['rms'], val_features['mfcc']), axis=1)

validation_features.shape

(2928, 81)

In [43]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [44]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_rms_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_rms_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 0.8847099535561425
Validation accuracy: 0.40027322404371585


## **With Mel-Spectrogram, MFCCs, Chroma**

In [45]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(10981, 92)

In [46]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(2928, 92)

In [47]:
model = SVC(kernel='rbf', C=10)
model.fit(training_features, train_y)

SVC(C=10)

In [48]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 0.8644932155541389
Validation accuracy: 0.41154371584699456


# Review results from all models

In [49]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,0.865,0.412
6,melspectrogram_chroma_mfcc,0.864,0.412
5,melspectrogram_rms_mfcc,0.885,0.400
1,mfcc,0.588,0.385
3,mfcc_chroma,0.661,0.377
2,mfcc_melspectrogram,0.884,0.399
4,mfcc_rms,0.595,0.398
